# KB Mirrors

This example simulates the alignment of a KB mirror endstation (with four degrees of freedom).

In [ ]:
import datetime

import matplotlib.pyplot as plt
import numpy as np
from bluesky.callbacks import best_effort
from bluesky.callbacks.tiled_writer import TiledWriter
from bluesky.run_engine import RunEngine
from ophyd.utils import make_dir_tree
from tiled.client import from_uri
from tiled.server import SimpleTiledServer

from blop.sim import beamline

tiled_server = SimpleTiledServer(readable_storage=[beamline.DECTECTOR_STORAGE])
tiled_client = from_uri(tiled_server.uri)
tiled_writer = TiledWriter(tiled_client)


def setup_re_env(db_type="default", root_dir="/default/path"):
    RE = RunEngine({})
    bec = best_effort.BestEffortCallback()
    RE.subscribe(bec)
    RE.subscribe(tiled_writer)
    _ = make_dir_tree(datetime.datetime.now().year, base_path=root_dir)
    return {"RE": RE, "tiled": tiled_client, "bec": bec}


env = setup_re_env(db_type="temp", root_dir="/tmp/blop/test/sim")
globals().update(env)

bec.disable_plots()

In [ ]:
# from blop.utils import prepare_re_env
#from blop.utils import prepare_re_env  # noqa

# %run -i $prepare_re_env.__file__ --db-type=temp
# bec.disable_plots()

In [ ]:
from blop.sim import Beamline

beamline = Beamline(name="bl")

In [ ]:
from blop import DOF, Agent, Objective
from blop.digestion import beam_stats_digestion

dofs = [
    DOF(description="KBV downstream", device=beamline.kbv_dsv, search_domain=(-5.0, 5.0)),
    DOF(description="KBV upstream", device=beamline.kbv_usv, search_domain=(-5.0, 5.0)),
    DOF(description="KBH downstream", device=beamline.kbh_dsh, search_domain=(-5.0, 5.0)),
    DOF(description="KBH upstream", device=beamline.kbh_ush, search_domain=(-5.0, 5.0)),
]

objectives = [
    Objective(name="bl_det_sum", target="max", transform="log", trust_domain=(200, np.inf)),
    Objective(name="bl_det_wid_x", target="min", transform="log", latent_groups=[("bl_kbh_dsh", "bl_kbh_ush")]),
    Objective(name="bl_det_wid_y", target="min", transform="log", latent_groups=[("bl_kbv_dsv", "bl_kbv_usv")]),
]

agent = Agent(
    dofs=dofs,
    objectives=objectives,
    detectors=[beamline.det],
    digestion=beam_stats_digestion,
    digestion_kwargs={"image_key": "bl_det_image"},
    verbose=True,
    tiled=tiled,
    tolerate_acquisition_errors=False,
    enforce_all_objectives_valid=True,
    train_every=3,
)
(uid,) = RE(agent.learn("qr", n=32))

In [ ]:
RE(agent.learn("qei", n=4, iterations=4))

In [ ]:
agent.plot_objectives(axes=(2, 3))

In [ ]:
plt.imshow(agent.best.bl_det_image)

In [ ]:
agent.plot_objectives(axes=(2, 3))

In [ ]:
tiled_server.close()

In [ ]:
tiled_server.close()